In [1]:
# coinmarketcap api to tap into small cap coin screener
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import streamlit as st
from pandas import json_normalize


In [2]:
# https://coinmarketcap.com/api/documentation/v1/#operation/getV1CryptocurrencyListingsLatest
api = "72bff5eb-7918-4881-aeb1-ef6addf6027e"

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'3000',
  'limit':'4000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': api,
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
    df = pd.DataFrame(data['data'])
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [123]:
# df.sort_values(by='date_added', ascending=False).head(1).transpose()
# df.explode('quote').reset_index(drop=True)
platform = json_normalize(df['platform'])
platform.columns = ['platform_id', 'platform_name', 'platform_symbol', 'platform_slug', 'platform_token_address']
df_all = pd.concat([df, platform, json_normalize(df['quote'])], axis=1)
df.drop(columns=['quote', 'platform'], inplace=True)
df_all.to_csv("top5000_cmc.csv")

In [4]:
df_all = pd.read_csv("top5000_cmc.csv")
df.head()

,Unnamed: 0,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,...,USD.percent_change_24h,USD.percent_change_7d,USD.percent_change_30d,USD.percent_change_60d,USD.percent_change_90d,USD.market_cap,USD.market_cap_dominance,USD.fully_diluted_market_cap,USD.tvl,USD.last_updated
0,0,29280,META PLUS TOKEN,MTS,meta-plus-token,4,2024-02-05T07:37:10.000Z,['polygon-ecosystem'],2.500000e+08,0,...,-2.538595,-1.946998,-13.092678,-13.092678,-13.092678,0,0.0,5874165.07,NaN,2024-02-23T00:21:00.000Z
1,1,10093,Gold Secured Currency,GSX,gold-secured-currency,6,2021-05-27T00:00:00.000Z,['commodities'],3.000000e+10,0,...,-9.137948,-0.154006,11.129367,42.706356,11.004221,0,0.0,29984292.15,NaN,2024-02-23T00:21:00.000Z
2,2,9537,EpiK Protocol,AIEPK,epik-protocol,18,2021-05-03T00:00:00.000Z,"['ai-big-data', 'storage', 'dwf-labs-portfolio']",1.000000e+09,0,...,0.547865,50.276034,68.237702,49.120639,3.714075,0,0.0,8241350.89,NaN,2024-02-23T00:22:00.000Z
3,3,18699,Shade Protocol,SHD,shade-protocol,5,2022-03-10T06:15:34.000Z,[],NaN,0,...,34.543955,57.016572,67.507554,117.725780,208.714452,0,0.0,92182400.91,NaN,2024-02-23T00:22:00.000Z
4,4,23174,Storepay,SPC,storepay,4,2023-01-06T09:53:11.000Z,[],1.601267e+11,0,...,-1.834257,-4.673300,14.180507,12.109303,8.476572,0,0.0,10521241.64,NaN,2024-02-23T00:21:00.000Z


In [5]:
df_all.iloc[2]

Unnamed: 0                                                                          2
id                                                                               9537
name                                                                    EpiK Protocol
symbol                                                                          AIEPK
slug                                                                    epik-protocol
num_market_pairs                                                                   18
date_added                                                   2021-05-03T00:00:00.000Z
tags                                 ['ai-big-data', 'storage', 'dwf-labs-portfolio']
max_supply                                                               1000000000.0
circulating_supply                                                                  0
total_supply                                                              400000000.0
platform                            {'id': 1027, 'name

In [33]:
# look at the most frequent tags
all_tags_list = [ele[1:-1].replace("'", "").split(", ") for ele in df_all.tags if ele != '[]']
all_tags = [item for sublist in all_tags_list  for item in sublist]
tags = pd.DataFrame({'tag':all_tags}).drop_duplicates()
# tags['coins'] = [sum(df_all.tags.apply(lambda x: any(i in item for item in x))) for i in tags.tag]
tags['coins'] = [sum(df_all.tags.apply(lambda x: i in x))for i in tags.tag]
tags.sort_values(by='coins', ascending=False, inplace=True)
tags.to_csv("count_by_tags.csv")
tags.head(20)

,tag,coins
5,memes,524
32,bnb-chain,233
38,mineable,157
7,gaming,146
6,collectibles-nfts,140
44,ethereum-ecosystem,119
39,pow,115
1165,art,113
19,defi,111
71,solana-ecosystem,99


In [34]:
# look at number of coins within each chain
df_all.groupby("platform_slug").symbol.count().sort_values(ascending=False).head(20)

platform_slug
ethereum           1638
bnb                1555
solana              237
polygon             208
arbitrum            110
avalanche            73
fantom               57
tron                 43
klaytn               38
bitcicoin            29
cardano              29
cronos               26
multiversx-egld      24
zksync               24
osmosis              21
toncoin              20
bitcoin              19
base                 19
pulsechain           17
aptos                17
Name: symbol, dtype: int64

In [36]:
# find all coins with certain tag: 
i = "distributed-computing"
# theme = df_all[df_all.tags.apply(lambda x: any(i in item for item in x))]
theme = df_all[df_all.tags.apply(lambda x: i in x)]
theme.to_csv(i + "_coins.csv")
theme

,Unnamed: 0,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,...,USD.percent_change_24h,USD.percent_change_7d,USD.percent_change_30d,USD.percent_change_60d,USD.percent_change_90d,USD.market_cap,USD.market_cap_dominance,USD.fully_diluted_market_cap,USD.tvl,USD.last_updated
224,224,3126,Sirius Chain,XPX,proximax,11,2018-08-08T00:00:00.000Z,"['platform', 'enterprise-solutions', 'distribu...",NaN,0,...,32.456826,50.125859,80.091451,134.731269,95.624714,0,0.0000,1.099989e+07,NaN,2024-02-23T00:22:00.000Z
262,262,24067,krest Network,KREST,krest-network,2,2023-08-02T20:58:45.000Z,"['distributed-computing', 'polkadot-ecosystem']",4.000000e+08,0,...,0.417766,-2.747954,-13.403810,72.913825,25.059185,0,0.0000,1.425897e+08,NaN,2024-02-23T00:21:00.000Z
695,695,3863,UGAS,UGAS,ugas,8,2019-04-16T00:00:00.000Z,"['platform', 'distributed-computing']",NaN,0,...,0.502342,-0.395037,-17.300851,-3.397287,12.250659,0,0.0000,7.332572e+05,NaN,2024-02-23T00:21:00.000Z
721,721,2882,Zus,ZCN,0chain,13,2018-07-02T00:00:00.000Z,"['platform', 'ai-big-data', 'distributed-compu...",4.000000e+08,0,...,2.936299,-10.013840,-6.214287,-4.912764,46.968762,0,0.0000,8.709680e+07,NaN,2024-02-23T00:21:00.000Z
909,909,2392,Bottos,BTO,bottos,3,2018-01-14T00:00:00.000Z,"['marketplace', 'ai-big-data', 'distributed-co...",NaN,0,...,-8.163007,21.874048,17.409641,57.605516,108.584069,0,0.0000,0.000000e+00,NaN,2024-02-23T00:22:00.000Z
939,939,27319,Precipitate.AI,RAIN,precipitate-ai,6,2023-06-30T07:24:26.000Z,['distributed-computing'],1.000000e+09,0,...,12.301826,78.687049,-8.779455,138.073596,428.911679,0,0.0000,4.098091e+06,NaN,2024-02-23T00:21:00.000Z
1124,1124,8593,FileStar,STAR,filestar,5,2021-02-25T00:00:00.000Z,"['distributed-computing', 'filesharing', 'stor...",2.000000e+09,0,...,11.441511,5.127944,-1.009030,12.994508,61.503692,0,0.0000,1.716669e+07,NaN,2024-02-23T00:22:00.000Z
1315,1315,20969,GEODNET,GEOD,geodnet,30,2022-07-12T13:42:17.000Z,['distributed-computing'],1.000000e+09,0,...,0.715125,14.556734,-2.325509,-29.780990,47.198413,0,0.0000,1.082792e+08,NaN,2024-02-23T00:21:00.000Z
1430,1430,2535,Edge,EDGE,edge,2,2018-02-19T00:00:00.000Z,"['platform', 'distributed-computing', 'filesha...",6.000000e+07,0,...,-0.249413,-10.580251,-25.466220,106.218342,39.884084,0,0.0004,1.020054e+07,NaN,2024-02-23T00:21:00.000Z
3681,3681,291,MaidSafeCoin,MAID,maidsafecoin,4,2014-04-28T00:00:00.000Z,"['platform', 'distributed-computing', 'filesha...",4.525524e+08,0,...,1.225883,6.156137,-19.331132,2.719702,102.307766,0,0.0000,7.739976e+07,NaN,2024-02-23T00:22:00.000Z


In [6]:
# import configparser
# import json
# import pytz
# from datetime import datetime
# from requests import Request, Session
# from dateutil import parser

# def get_info():
#     # Read the API key from the coinmarket.ini file
#     config = configparser.ConfigParser()
#     config.read('coinmarket.ini')
#     # api_key = config['DEFAULT']['API_KEY']
#     api_key = api
    
#     # Set up the request parameters and headers
#     url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest'
#     parameters = { 'slug': 'bitcoin', 'convert': 'USD' }
#     headers = {
#         'Accepts': 'application/json',
#         'X-CMC_PRO_API_KEY': api_key
#     }

#     # Send the request and retrieve the response
#     session = Session()
#     session.headers.update(headers)
#     response = session.get(url, params=parameters)
#     info = json.loads(response.text)

#     # Extract the desired information from the response
#     data = info['data']['1']
#     name = data['name']
#     symbol = data['symbol']
#     rank = data['cmc_rank']
#     total_supply = data['total_supply']
#     circulating_supply = data['circulating_supply']
#     market_cap = data['quote']['USD']['market_cap']
#     price = data['quote']['USD']['price']
#     market_cap_dominance = data['quote']['USD']['market_cap_dominance']
#     percent_change_1h = data['quote']['USD']['percent_change_1h']
#     percent_change_24h = data['quote']['USD']['percent_change_24h']
#     volume_24h = data['quote']['USD']['volume_24h']
#     volume_change_24h = data['quote']['USD']['volume_change_24h']
#     timestamp = info['status']['timestamp']

#     # Convert the timestamp to a timezone-aware datetime object
#     timestamp_local = parser.parse(timestamp).astimezone(pytz.timezone('Turkey'))
    

#     # Format the timestamp as desired
#     formatted_timestamp = timestamp_local.strftime('%Y-%m-%d %H:%M:%S')

#     # Print the information
#     print(f'Name: {name}, Symbol: {symbol}, Price: {price:,.2f}, Percent change (1h): {percent_change_1h}, Percent change (24h): {percent_change_24h}, Total supply: {total_supply}, Circulating supply: {circulating_supply}, Market capitalization: {market_cap}, Market capitalization dominance: {market_cap_dominance}, Volume (24h): {volume_24h}, Volume change (24h): {volume_change_24h}, Timestamp: {formatted_timestamp}')

# get_info()

Name: Bitcoin, Symbol: BTC, Price: 48,602.89, Percent change (1h): 0.68029867, Percent change (24h): 1.88871167, Total supply: 19625225, Circulating supply: 19625225, Market capitalization: 953842679372.7578, Market capitalization dominance: 52.4813, Volume (24h): 20623932898.459293, Volume change (24h): 27.4978, Timestamp: 2024-02-12 03:55:48
